# **Overview**
The LLama 2 project is a collection of pretrained and fine-tuned generative text models designed specifically for dialogue use cases. Ranging from 7 billion to 70 billion parameters, these models outperform open-source chat models on various benchmarks and demonstrate comparable performance to popular closed-source models in terms of helpfulness and safety.

## **LLama 2 13B-chat Model**
The llama.cpp file within this repository serves the objective of running the LLaMA model with 4-bit integer quantization on MacBook. It is a C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Initially developed as a web chat example, it now acts as a development playground for ggml library features.

## **GGML Library**
The GGML (Generative Generative Models Library) is a C library for machine learning. It facilitates the distribution of large language models (LLMs) and uses quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version numbers, hyperparameters, vocabulary, and weights. The vocabulary includes tokens for language generation, and the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

## **Quantized Models from Hugging Face Community**
The Hugging Face community provides quantized models that efficiently utilize the model on the T4 GPU. Ensure to consult reliable sources before using any model. Among the available variations, we are interested in those based on the GGLM library.

The Llama-2-13B-GGML model, and its variations, can be found [here](https://huggingface.co/models?search=llama%202%20ggml).


# **Installation**

## Step 1: Install Required **Packages**

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


## Step 2: Import Required **Libraries**

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama


## Step 3: Download the **Model**

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)


## Step 4: Load the **Model**

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32  # Change this value based on your model and your GPU VRAM pool.
)

## Step 5: Load Test **Dataset**

In [ ]:
from google.colab import drive
import json

# Mount Google Drive
drive.mount('/content/drive')

# Function to load the test dataset from Google Drive
def load_test_dataset(file_path):
    with open(file_path, "r") as file:
        dataset = [json.loads(line) for line in file]
    return dataset

# Load the test dataset
test_dataset_path = "/content/drive/MyDrive/test.jsonl"  # Update with your actual path
test_data = load_test_dataset(test_dataset_path)


## Step 6: Generate **Predictions**

In [ ]:
def generate_predictions(prompt):
    response = lcpp_llm(prompt=prompt, max_tokens=256, temperature=0.5, top_p=0.95, repeat_penalty=1.2, top_k=150, echo=True)
    return response

# Generate predictions for each text in the test dataset
predicted_labels = []
for example in test_data[:300]:
    text = ' '.join(example['text'])[:512]
    prompt = f"predict the relevant statutes {text}"
    response = generate_predictions(prompt)
    response_text = response['choices'][0]['text']
    predicted_label = response_text.splitlines()[0]
    predicted_labels.append(predicted_label)

## Step 7: Evaluate **Performance**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

# Function to evaluate the model
def evaluate_model(predictions, true_labels):
    mlb = MultiLabelBinarizer()
    true_labels_binary = mlb.fit_transform(true_labels)
    predictions_binary = mlb.transform(predictions)

    accuracy = accuracy_score(true_labels_binary, predictions_binary)
    precision = precision_score(true_labels_binary, predictions_binary, average="macro")
    recall = recall_score(true_labels_binary, predictions_binary, average="macro")
    f1 = f1_score(true_labels_binary, predictions_binary, average="macro")

    return accuracy, precision, recall, f1

# Extract true labels from the test dataset
true_labels = [example['labels'][0] for example in test_data[:300]]

# Evaluate the model
accuracy, precision, recall, f1 = evaluate_model(predicted_labels, true_labels)

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Macro Precision: {precision}")
print(f"Macro Recall: {recall}")
print(f"Macro F1: {f1}")
